# NBA Bet Prop Analysis

## Part 2 - Updates 

This script is meant to handle the daily data updates to the SQLite database (PropAnalysis.db). Just like in Part 1, this script will scrape DraftKings for NBA bet odds, SportsLine for the NBA projections, and RapidAPI's API-NBA feed for the actual game stats. However, in this script we will not be replacing the table, but appending to it. 

During this update process we will also be able to "score" our bets to see if they won or lost. After collecting a good sample size of results, we will be able to compare our win rate to our estimated edge to see if their is a good correlation that would imply a profitable betting model. 

In [175]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import json
from datetime import date, datetime, timedelta
from pytz import timezone
import scipy.stats
import sqlite3
from sqlalchemy import create_engine
import time

### Helper Functions 

In [77]:
def american2DecimalOdds(americanBetOdds):
    """
    @betOdds (str) the American odds of the bet. Must be prefaced with a '+' or '-'
    
    Returns the bet odds in decimal format. 
    """
    try:
        if (americanBetOdds[0] == '+'):
            americanBetOdds = int(americanBetOdds[1:])
            decimalOdds = (americanBetOdds/100) + 1
            # Return the decimalOdds in a clean format
            return(round(decimalOdds,2))
        elif americanBetOdds[0] == '-':
            americanBetOdds = int(americanBetOdds[1:])
            decimalOdds = (100/americanBetOdds) + 1
            # Return the decimalOdds in a clean format
            return(round(decimalOdds,2))
        else:
            print("Bet odds must begin with a '+' or '-'")
    except:
        return(None)

In [78]:
def impliedOdds(betOdds):
    """
    @betOdds (str) the American odds of the bet. Must be prefaced with a '+' or '-'
    
    Takes in the betOdds and returns the implied probability of the bet
    """
    try:
        # First need to convert the American odds to decimal odds
        decimalOdds = american2DecimalOdds(betOdds)

        # Use the decimal odds to return the implied probability
        probability = 1/decimalOdds * 100

        # Return the probability rounded to the nearest whole number
        return(round(probability))
    except:
        return(None)

In [79]:
def findTeamIds(Game):
    """
    @Game str: the DraftKings listed game
    Example: "GS Warriors @ NO Pelicans"
    
    Return: @homeTeamId int,
            @awayTeamId int
    """
    # Replace certain names from DraftKings to Match our NBA_Teams table
    Game = Game.replace("GS Warriors", "GSW Warriors").replace("SA Spurs", "SAS Spurs")
    Game = Game.replace("NO Pelicans", "NOP Pelicans").replace("LA Clippers", "LAC Clippers")
    Game = Game.replace("NY Knicks", "NYK Knicks").replace("LA Lakers", "LAL Lakers")
    Game = Game.replace("Trail Blazers", "TrailBlazers").replace("PHO Suns", "PHX Suns")
    # Split the game into home and away team
    aTeam, hTeam = Game.split("@")
    # Clean the whitespaces from either end
    hTeam = hTeam.strip()
    aTeam = aTeam.strip()
    # Split the teams in the City and Nickname
    hShortName, hNickname = hTeam.split(" ")
    aShortName, aNickname = aTeam.split(" ")
    # Replace certain names from DraftKings to Match our NBA_Teams table
    hNickname = hNickname.replace("TrailBlazers", "Trail Blazers")
    aNickname = aNickname.replace("TrailBlazers", "Trail Blazers")
    # Use the City and Nickname to find the teamId
    hTeamId = NBA_Teams.loc[(NBA_Teams['shortName'] == hShortName) & (NBA_Teams['Nickname'] == hNickname), "teamId"].values[0]
    aTeamId = NBA_Teams.loc[(NBA_Teams['shortName'] == aShortName) & (NBA_Teams['Nickname'] == aNickname), "teamId"].values[0]
    # Return the results
    return(hTeamId, aTeamId)

In [80]:
def findGameId(Game, startDate, startTime):
    """
    @Game str: the DraftKings listed game. Example: 'GS Warriors @ NO Pelicans'
    @startDate str: the DraftKings listed game date. Example: 'Fri, Jan 14'
    @startTime str: the DraftKings listed game start time. Example: '07:30 PM'
    """
    # Get the home and awaw team ids
    hTeamId, aTeamId = findTeamIds(Game)
    # Get the current Year
    currentDateTime = datetime.now()
    date = currentDateTime.date()
    year = date.strftime("%Y")
    # Format startDate as yyyy-mm-dd format
    startDate = datetime.strptime(startDate, '%a, %b %d')
    startDate = startDate.strftime(f'{year}-%m-%d')
    # Format startTime hh:mm
    startTime = datetime.strptime(startTime, '%I:%M %p')
    startTime = startTime.strftime('%H:%M')
    # Use the startDate and startTime to find the gameId
    gameId = NBA_Games.loc[(NBA_Games['startDate'] == startDate) & (NBA_Games['homeTeamId'] == hTeamId) &
                           (NBA_Games['awayTeamId'] == aTeamId), "gameId"].values[0]
    # Return the results
    return(gameId)

In [171]:
def findPlayerStatStdDevs(Player, BetLabel):
    """
    @Player str, the name of the player. Ex: Kevon Looney
    @BetLabel str, the DraftKings listed bet name. Ex: Points + Assists + Rebound
    """
    # Find Player Id 
    try:
        playerId = NBA_Players.loc[NBA_Players["fullName"] == Player, "playerId"].values[0]
    except IndexError:
        return('NA')

    # Filter dataset to Giannis Antetokounmpo 
    df = NBA_GameStats[NBA_GameStats['playerId'] == 20]

    # Split betLabel into component parts, if possible  
    bet_parts = betLabel.split("+")
    bet_parts = [x.strip() for x in bet_parts]
    bet_parts.sort()

    # Link the NBA stat fields with the bet label tracked
    stat_label_map = {
        'Assists': 'assists',
        'Points': 'Points',
        'Rebounds': 'totReb'
    }
    stat_label_keys = list(stat_label_map.keys())

    # Extract the std_dev of NBA Stats from the NBA_GameStats table based off of the betLabel
    if len(bet_parts) == 3:

        # Check that the labels are those in the stat_label_map 
        # If not this is likely due to a miss in the Player matching that results in overlap in the BetLabel
        if list(bet_parts) == stat_label_keys:

            # Create a variable to store the NBA Stat
            std_dev = 0

            # Loop over the bet parts, find the standard deviation, and add them
            for part in bet_parts:

                # Find std deviations for stats of interest (Points, Assists, Rebounds)
                s = round(df.loc[:,stat_label_map[part]].std(),1)
                std_dev += s


    elif len(bet_parts) == 2:

        # Check that both the labels are in the stat_label_lookup
        # If not this is likely due to a miss in the Player matching that results in overlap in the BetLabel
        if (bet_parts[0] in stat_label_keys) & (bet_parts[1] in stat_label_keys) :

            # Create a variable to store the NBA Stat
            std_dev = 0

            # Loop over the bet parts, find the standard deviation, and add them
            for part in bet_parts:

                # Find std deviations for stats of interest (Points, Assists, Rebounds)
                s = round(df.loc[:,stat_label_map[part]].std(),1)
                std_dev += s

    else:

        if betLabel in stat_label_map.keys():

                # Find std deviations for stats of interest (Points, Assists, Rebounds)
                std_dev = round(df.loc[:,stat_label_map[part]].std(),1)
    
    return(std_dev)

In [172]:
def findEstimatedEdge(SportsLine_Projection, DK_Line, Std_Dev, Over_Odds, Under_Odds):
    """
    @SportsLine_Projection float
    @DK_Line float
    @StdDev float
    @Over_Odds text
    @Under_Odds text
    """
    # Find the edge if a Std_Dev was found
    if Std_Dev != 'NA':
        
        # Find the odds of going over and under the number
        cdf = scipy.stats.norm(SportsLine_Projection, Std_Dev).cdf(DK_Line)
        over_prob = round((1 - cdf) * 100,1)
        under_prob = round(cdf * 100,1)

        # Convert the odds to implied probability
        over_implied_odds = impliedOdds(over_odds)
        under_implied_odds = impliedOdds(under_odds)

        # Find the expected edge by comparing the probabilities with the implied odds
        over_expected_edge = round(over_prob - over_implied_odds,1)
        under_expected_edge = round(under_prob - under_implied_odds,1)

        # Return the larger edge value
        if over_expected_edge > under_expected_edge:
            return(over_expected_edge)
        else:
            return(under_expected_edge)
        
    else:
        
        return('NA')

In [174]:
def betGrade(EstimatedEdge):
    """
    @EstimatedEdge float: the estimated edge the bet has versus the DraftKings bet line given the SportsLine projection,
    the bet odds, and the player's standard deviations for the bet labels
    """
    if EstimatedEdge > 20:
        grade = 'A'
    elif EstimatedEdge > 15:
        grade = 'B'
    elif EstimatedEdge > 10:
        grade = 'C'
    elif EstimatedEdge > 5:
        grade = 'D'
    else:
        grade = 'F'
    return(grade)

## Pull in SQLite Tables and Create Backups

In [158]:
# Connect to SQLite
conn = sqlite3.connect('Data/PropAnalysis.db')

# Query NBA_Teams table into dataframe
NBA_Teams = pd.read_sql_query("SELECT * FROM NBA_Teams", conn)
# Save the file as a csv for analysis
NBA_Teams.to_csv('Data/Backup/NBA_Teams.csv', index=False)

# Query NBA_Players table into dataframe
NBA_Players = pd.read_sql_query("SELECT * FROM NBA_Players", conn)
# Save the file as a csv for analysis
NBA_Players.to_csv('Data/Backup/NBA_Players.csv', index=False)

# Query NBA_Games table into dataframe
NBA_Games = pd.read_sql_query("SELECT * FROM NBA_Games", conn)
# Save the file as a csv for analysis
NBA_Games.to_csv('Data/Backup/NBA_Games.csv', index=False)

# Query NBA_GameStats table into dataframe
NBA_GameStats = pd.read_sql_query("SELECT * FROM NBA_GameStats", conn)
# Save the file as a csv for analysis
NBA_GameStats.to_csv('Data/Backup/NBA_GameStats.csv', index=False)

# Query bet_offers_df table into dataframe
bet_offers_df = pd.read_sql_query("SELECT * FROM bet_offers_df", conn)
# Save the file as a csv for analysis
bet_offers_df.to_csv('Data/Backup/BetOdds.csv', index=False)

# Query SportsLine_Projections table into dataframe
SportsLine_Projections = pd.read_sql_query("SELECT * FROM SportsLine_Projections", conn)
# Save the file as a csv for analysis
SportsLine_Projections.to_csv('Data/Backup/SportsLine_Projections.csv', index=False)

# Close connection when done
conn.close()

## Pull Today's Data 

In [83]:
# Returns the current local date
today = date.today()
print("Today date is: ", today)

Today date is:  2022-01-18


### DraftKings - NBA

In [176]:
# Create a dataframe to append results to
bet_offers_dailydf = pd.DataFrame(columns=['League', 'Game', 'StartDate', 'StartTime', 'Player', 'BetLabel', 
                                           'SportsLine_Projection', 'DK_Line', 'Over_Odds', 'Under_Odds'])

# Create a list of the DraftKings URLs to parse
url = "https://sportsbook.draftkings.com/leagues/basketball/88670846"
    
# Get the BeautifulSoup data from the DraftKings website
dk_response = requests.get(url)
dk_soup = BeautifulSoup(dk_response.text, "html.parser")

# Narrow the Beautiful Soup extract to just the field of interest "window.__INITIAL_STATE__"

# Filter out opening and closing <script> tags
dk_scrape = str(list(list(list(dk_soup.children)[2])[3])[11]).replace("<script>","").replace("</script>", "")
# Remove leading and trailing whitespace
dk_scrape = dk_scrape.strip()
# Split sections
dk_scrape = dk_scrape.split(";\n")
# Isolate to json of interest
dk_scrape = dk_scrape[6].strip()
# Format as json dictionary
dk_scrape = dk_scrape.replace("window.__INITIAL_STATE__ = ","")
dk_scrape = json.loads(dk_scrape)

# Grab the sport ID from the scrape
sportId = list(dk_scrape['eventGroups'].keys())[0]

# From the full scrape of the page, pull a list of the games to loop over and extract data from
games = dk_scrape['eventGroups'][sportId]['events'].keys()

# Loop over games
for index, game in enumerate(games):

    # Set the game JSON as variable
    game_details = dk_scrape['eventGroups'][sportId]['events'][game]
    # Get the eventId so I can scrape the actual props
    eventId = game_details['eventId']

    # Web scrape Draft Kings for player props
    props_url = f"https://sportsbook.draftkings.com/event/{eventId}"
    response = requests.get(props_url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Clean BeautifulSoup response
    # Filter out opening and closing <script> tags
    scrape = str(list(list(list(soup.children)[2])[3])[11]).replace("<script>","").replace("</script>", "")
    # Remove leading and trailing whitespace
    scrape = scrape.strip()
    # Split sections
    scrape = scrape.split(";\n")
    # Isolate to json of interest
    scrape = scrape[6].strip()
    # Format as json dictionary
    scrape = scrape.replace("window.__INITIAL_STATE__ = ","")
    scrape = json.loads(scrape)

    # Parse the scrape results
    eventGroupId = list(scrape['eventGroups'].keys())[0]
    providerEventId = list(scrape['eventGroups'][eventGroupId]['events'].keys())[0]
    providerOfferId = list(scrape['offers'][eventGroupId].keys())[0]
    eventId = scrape['eventGroups'][eventGroupId]['events'][providerEventId]['eventId']
    game_details = scrape['eventGroups'][eventGroupId]['events'][providerEventId]
    game = game_details['name']
    eventGroup = game_details['eventGroupName']
    teamName1 = game_details['teamName1']
    teamName1 = game_details['teamName2']
    startDate = game_details['startDate']
    date, dtime = startDate.split('T')
    dt = date + ' ' + dtime[:8]
    dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
    # Format as right timezone (-5 hours)
    dt = dt - timedelta(hours=5)
    date = dt.strftime('%a, %b %d')
    dtime = dt.strftime('%I:%M %p')

    # Isolate the bet offers from the beautiful soup scrape
    offers = scrape['offers'][eventGroupId]

    # Loop over offers and add to dataframe
    for index, offer in enumerate(offers):

        # Create a list for parsing the offers 
        offers_parsed = [eventGroup, game, date, dtime, '', '', '', '', '', '']

        # Parse the betting offer
        offer_dict = scrape['offers'][eventGroupId][offer]
        try:
            providerOfferId = offer_dict['providerOfferId']
            providerId = offer_dict['providerId']
            providerEventId = offer_dict['providerEventId']
            bet_label = offer_dict['label']
            isOpen = offer_dict['isOpen']
            outcomes = offer_dict['outcomes']
        except:
            continue

        # Assign to list
        offers_parsed[5] = bet_label    

        # Extract outcomes for two-side bets
        if len(outcomes) == 2:

            # Parse the outcomes
            for i, x in enumerate(outcomes):
                if i == 0:
                    outcome_label = outcomes[i]['label']
                    try:
                        outcome_line = outcomes[i]['line']
                    except:
                        outcome_line = ''
                    outcome_odds = outcomes[i]['oddsAmerican']
                    offers_parsed[7] = outcome_line
                    offers_parsed[8] = outcome_odds
                else:
                    outcome_label = outcomes[i]['label']
                    try:
                        outcome_line = outcomes[i]['line']
                    except:
                        continue
                    outcome_odds = outcomes[i]['oddsAmerican']
                    offers_parsed[9] = outcome_odds

            # Append the list to the dataframe 
            bet_offers_dailydf.loc[len(bet_offers_dailydf)] = offers_parsed

        else:

            continue

# Preview output
bet_offers_dailydf.head()

,League,Game,StartDate,StartTime,Player,BetLabel,SportsLine_Projection,DK_Line,Over_Odds,Under_Odds
0,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,,Hamidou Diallo Turnovers,,1.5,+105,-145
1,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,,Hamidou Diallo Assists + Rebounds,,6.5,-135,+100
2,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,,Hamidou Diallo Blocks,,0.5,+140,-195
3,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,,Hamidou Diallo Steals,,1.5,+145,-205
4,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,,Hamidou Diallo Points + Rebounds,,17.5,-120,-115


### SportsLine Projections - NBA 

In [85]:
# Web scrape Fantasy Pros for relevant information
url = 'https://www.sportsline.com/nba/expert-projections/simulation/'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Isolate soup to the table of interest
output = str(list(list(list(soup.children)[1])[1])[1])
# Format as json dictionary
output = output.replace("""<script id="__NEXT_DATA__" type="application/json">""","").replace("</script>","")
output = json.loads(output)
projections = output['props']['initialState']['fantasyState']['projectionsPageState']['data']['projections']

# Create a dataframe to append results to
NBA_SportsLine_Projections = pd.DataFrame(columns=['LEAGUE', 'PLAYER', 'POS', 'TEAM', 'GAME', 'FP', 'PTS', 'MIN', 'FG',
                                                   'FGA', 'AST', 'TRB', 'DRB', 'ORB', 'BK', 'ST', 'TO', 'FT', 'FTP', 
                                                   'FGP', 'DATE'])
                                                   
# Loop over the projections and parse
for i, projection in enumerate(projections):
    # Create an empty list to update the data for
    player_projections = [0] * 21
    # Set LEAGUE columns
    player_projections[0] = 'NBA'
    # Loop over list of projections to scrape
    projectionFields = projection['projectionFields']
    # Create a counter to add additional fields to correct list position
    counter = 1
    for x in projectionFields:
        field = x['field']
        if field in NBA_SportsLine_Projections.columns:
            try:
                value = x['value']
                player_projections[counter] = value
            except KeyError:
                player_projections[counter] = 0
            counter += 1
    # Set DATE column with today's date
    player_projections[20] = today
    # Append the list to the dataframe         
    NBA_SportsLine_Projections.loc[len(NBA_SportsLine_Projections)] = player_projections

# Preview output
NBA_SportsLine_Projections.head()

,LEAGUE,PLAYER,POS,TEAM,GAME,FP,PTS,MIN,FG,FGA,...,TRB,DRB,ORB,BK,ST,TO,FT,FTP,FGP,DATE
0,NBA,Julius Randle,PF,NY,MIN@NY,38,20.6,35,7.5,16.6,...,10,8.4,1.6,0.5,0.8,3.8,3.7,0.76,0.45,2022-01-18
1,NBA,Karl-Anthony Towns,C,MIN,MIN@NY,37.7,22.3,34,8.1,15.8,...,9.1,6.8,2.2,1.3,0.8,3.0,3.9,0.82,0.51,2022-01-18
2,NBA,D`Angelo Russell,PG,MIN,MIN@NY,32.2,17.7,31,6.5,15.2,...,3.2,2.9,0.3,0.4,0.9,2.3,1.9,0.81,0.43,2022-01-18
3,NBA,Anthony Edwards,SG,MIN,MIN@NY,30.1,20.1,34,7.4,16.5,...,4.3,3.7,0.6,0.5,1.1,2.4,2.4,0.80,0.45,2022-01-18
4,NBA,Andrew Wiggins,SF,GS,DET@GS,30.1,19.1,33,7.1,13.8,...,4.7,3.8,0.9,0.8,1.0,1.9,2.3,0.70,0.51,2022-01-18


In [86]:
# Connect to SQLite
conn = sqlite3.connect('Data/PropAnalysis.db')

# Create the connection to the SQLite database
engine = create_engine('sqlite:///Data/PropAnalysis.db')
sqlite_connection = engine.connect()

# Update and replace the table in SQLite3
sqlite_table = "SportsLine_Projections"
NBA_SportsLine_Projections.to_sql(sqlite_table, sqlite_connection, if_exists='append')

# Close connection when done
sqlite_connection.close()
conn.close()

2022-01-18 14:02:11,517 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SportsLine_Projections")
2022-01-18 14:02:11,518 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-18 14:02:11,525 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-18 14:02:11,530 INFO sqlalchemy.engine.Engine INSERT INTO "SportsLine_Projections" ("index", "LEAGUE", "PLAYER", "POS", "TEAM", "GAME", "FP", "PTS", "MIN", "FG", "FGA", "AST", "TRB", "DRB", "ORB", "BK", "ST", "TO", "FT", "FTP", "FGP", "DATE") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-01-18 14:02:11,530 INFO sqlalchemy.engine.Engine [generated in 0.00146s] ((0, 'NBA', 'Julius Randle', 'PF', 'NY', 'MIN@NY', 38, 20.6, 35, 7.5, 16.6, 4.9, 10, 8.4, 1.6, 0.5, 0.8, 3.8, 3.7, 0.76, 0.45, '2022-01-18'), (1, 'NBA', 'Karl-Anthony Towns', 'C', 'MIN', 'MIN@NY', 37.7, 22.3, 34, 8.1, 15.8, 3.6, 9.1, 6.8, 2.2, 1.3, 0.8, 3.0, 3.9, 0.82, 0.51, '2022-01-18'), (2, 'NBA', 'D`Angelo Russell', 'PG', 'MIN', 'MIN@NY', 32.2, 17.7,

### Connect Bet Lines to Projections 

In [87]:
# Connect to SQLite
conn = sqlite3.connect('Data/PropAnalysis.db')

# Query SportsLine_Projections table into dataframe
SportsLine_Projections = pd.read_sql_query("SELECT * FROM SportsLine_Projections", conn)
# Drop duplicates from SporsLine_Projections table
SportsLine_Projections = SportsLine_Projections.drop_duplicates(keep='last')

# Close connection when done
conn.close()

In [88]:
# Get a list of the players in the projections tables
SportsLine_players = SportsLine_Projections['PLAYER'].to_list()

In [166]:
# For every row in bef_offers_df, check if the label contains the name of a player, if so add it to the Player column
for i, row in bet_offers_dailydf.iterrows():
    betLabel = row["BetLabel"]
    player = [x for x in SportsLine_players if x in betLabel]
    if len(player) > 0:
        # Extract the player name from the list
        player = player[0]
        # Update the Player field in the df
        bet_offers_dailydf.loc[i, "Player"] = player
        # Update the BetLabel to remove the player name to isolate the stat
        bet_offers_dailydf.loc[i, "BetLabel"] = betLabel.replace(player, "").strip()

In [167]:
# Link the SportsLine projection fields with the bet label tracked
stat_label_map = {
    'Assists': 'AST',
    'Points': 'PTS',
    'Rebounds': 'TRB'
}

stat_label_keys = list(stat_label_map.keys())

# Create a column for the SportsLine Projection
for i, row in bet_offers_dailydf.iterrows():
    # Collet needed variables
    BetLabel = row["BetLabel"]
    bet_parts = BetLabel.split("+")
    bet_parts = [x.strip() for x in bet_parts]
    bet_parts.sort()
    player = row["Player"]

    if bet_parts is not None:

        if len(bet_parts) == 3:

            # Check that the labels are those in the stat_label_map 
            # If not this is likely due to a miss in the Player matching that results in overlap in the BetLabel
            if list(bet_parts) == stat_label_keys:
        
                # Create a variable to store the NBA_SportsLine_Projection
                SportsLine_Projection = 0
                
                # Loop over the bet parts and add the projections
                for part in bet_parts:
                    # Use the player name and the statistic to look up the value in the NBA_SportsLine_Projections table
                    projection = SportsLine_Projections.loc[SportsLine_Projections.PLAYER == player, 
                                                                stat_label_map[part]].iloc[0]
                    # Add the projection to the total
                    SportsLine_Projection += projection
                    
                # Set the projection in the bet_offers_dailydf table
                bet_offers_dailydf.loc[i, "SportsLine_Projection"] = SportsLine_Projection

        elif len(bet_parts) == 2:
            
            # Check that both the labels are in the stat_label_lookup
            # If not this is likely due to a miss in the Player matching that results in overlap in the BetLabel
            if (bet_parts[0] in stat_label_keys) & (bet_parts[1] in stat_label_keys):
                
                # Create a variable to store the NBA_SportsLine_Projection
                SportsLine_Projection = 0
                
                # Loop over the bet parts and add the projections
                for part in bet_parts:
                    # Use the player name and the statistic to look up the value in the NBA_SportsLine_Projections table
                    projection = SportsLine_Projections.loc[SportsLine_Projections.PLAYER == player, 
                                                                stat_label_map[part]].iloc[0]
                    # Add the projection to the total
                    SportsLine_Projection += projection
                    
                # Set the projection in the bet_offers_dailydf table
                bet_offers_dailydf.loc[i, "SportsLine_Projection"] = SportsLine_Projection

        elif len(bet_parts) == 1:

            if BetLabel in stat_label_map.keys():
                # Use the player name and the statistic to look up the value in the NBA_SportsLine_Projections table
                SportsLine_Projection = SportsLine_Projections.loc[SportsLine_Projections.PLAYER == player, 
                                                                           stat_label_map[BetLabel]].iloc[0]
                # Set the projection in the bet_offers_dailydf table
                bet_offers_dailydf.loc[i, "SportsLine_Projection"] = SportsLine_Projection

        else:
            
            continue
    
    else:
        
        continue

# Format columns
bet_offers_dailydf['SportsLine_Projection'] = pd.to_numeric(bet_offers_dailydf['SportsLine_Projection'], errors='coerce')
bet_offers_dailydf['DK_Line'] = pd.to_numeric(bet_offers_dailydf['DK_Line'], errors='coerce')
bet_offers_dailydf['Outcome1_Label'] = bet_offers_dailydf['Outcome1_Label'].astype(str)
bet_offers_dailydf['Outcome2_Label'] = bet_offers_dailydf['Outcome2_Label'].astype(str)

# Add columns for difference between the projection and line in absolute values and percents
bet_offers_dailydf["Line2Proj_Diff"] = abs(bet_offers_dailydf['SportsLine_Projection'] - bet_offers_dailydf['DK_Line'])
bet_offers_dailydf["Line2Proj_PctDiff"] = abs((bet_offers_dailydf['SportsLine_Projection'] - bet_offers_dailydf['DK_Line']) 
                                              * 100 / bet_offers_dailydf['DK_Line'])

# Add an empty column for the NBA_Stat that will be filled in later
bet_offers_dailydf['NBA_Stat'] = 'NA'

# Drop rows which contain any NaN value in the selected columns
bet_offers_dailydf = bet_offers_dailydf.dropna(how='any', subset=['SportsLine_Projection', 'DK_Line'])
# Reset the index
bet_offers_dailydf.reset_index(inplace = True, drop = True)

# Preview output
bet_offers_dailydf.head()

,League,Game,StartDate,StartTime,Player,BetLabel,SportsLine_Projection,DK_Line,Outcome1_Label,Outcome1_Odds,Outcome2_Label,Outcome2_Odds,Line2Proj_Diff,Line2Proj_PctDiff,NBA_Stat
0,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,Hamidou Diallo,Assists + Rebounds,5.8,6.5,Over,-135,Under,+100,0.7,10.769231,NA
1,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,Hamidou Diallo,Points + Rebounds,14.7,17.5,Over,-120,Under,-115,2.8,16.000000,NA
2,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,Hamidou Diallo,Points + Assists,11.5,13.5,Over,-125,Under,-105,2.0,14.814815,NA
3,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,Cade Cunningham,Points + Assists,20.4,21.5,Over,-115,Under,-115,1.1,5.116279,NA
4,NBA,DET Pistons @ GS Warriors,"Tue, Jan 18",10:10 PM,Cade Cunningham,Assists + Rebounds,10.3,9.5,Over,-135,Under,+100,0.8,8.421053,NA


In [92]:
# Connect to SQLite
conn = sqlite3.connect('Data/PropAnalysis.db')

# Create the connection to the SQLite database
engine = create_engine('sqlite:///Data/PropAnalysis.db')
sqlite_connection = engine.connect()

# Update and replace the table in SQLite3
sqlite_table = "bet_offers_df"
bet_offers_dailydf.to_sql(sqlite_table, sqlite_connection, if_exists='append')

# Close connection when done
sqlite_connection.close()
conn.close()

2022-01-18 14:12:43,968 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bet_offers_df")
2022-01-18 14:12:43,969 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-18 14:12:43,973 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-18 14:12:43,976 INFO sqlalchemy.engine.Engine INSERT INTO bet_offers_df ("index", "League", "Game", "StartDate", "StartTime", "Player", "BetLabel", "SportsLine_Projection", "DK_Line", "Outcome1_Label", "Outcome1_Odds", "Outcome2_Label", "Outcome2_Odds", "Line2ProjDiff", "Outcome1_ImpliedProbability", "Outcome2_ImpliedProbability", "Bet_Juice", "NBA_Stat") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-01-18 14:12:43,977 INFO sqlalchemy.engine.Engine [generated in 0.00167s] ((0, 'NBA', 'DET Pistons @ GS Warriors', 'Tue, Jan 18', '10:10 PM', 'Hamidou Diallo', 'Assists + Rebounds', 5.8, 6.5, 'Over', '-135', 'Under', '+100', 0.7000000000000002, 57, 50.0, 57, 'NA'), (1, 'NBA', 'DET Pistons @ GS Warriors', 'Tue, Jan 18', '10:10 PM', 'Ha

### NBA Game Stats

#### Update NBA Games Table

In [93]:
# Hit RapidApi URL for NBA games for 2021 season
url = "https://api-nba-v1.p.rapidapi.com/games/seasonYear/2021"

headers = {
    'x-rapidapi-host': "api-nba-v1.p.rapidapi.com",
    'x-rapidapi-key': "40cfc5891cmshe48d38938873f74p12526ejsnb332fefe9905"
    }

response = requests.request("GET", url, headers=headers)

# Create a dataframe to append results to
NBA_Games = pd.DataFrame(columns=['gameId', 'seasonYear', 'League', 'homeTeamId', 'awayTeamId', 'startDate', 'startTime', 
                                  'status', 'homeScore', 'awayScore'])

# Set the response as a JSON and extract the games section
games = json.loads(response.text)
games = games['api']['games']

# Loop over the games and parse needed fields
for game in games:
    
    # Only care about NBA games (not G-league)
    league = game['league']
    
    if league == 'standard':
        
        # Grab needed fields
        seasonYear = game['seasonYear']
        gameId = game['gameId']
        startTimeUTC = game['startTimeUTC']
        arena = game['arena']
        city = game['city']
        status = game['statusGame']
        awayTeamId = game['vTeam']['teamId']
        homeTeamId = game['hTeam']['teamId']
        awayScore = game['vTeam']['score']['points']
        homeScore = game['hTeam']['score']['points']
        
        # Convert startTime to EST
        try:
            # Format string as datetime
            startTimeUTC = datetime.strptime(startTimeUTC, "%Y-%m-%dT%H:%M:%S.000Z")
            # Set UTC as timezone
            startTimeUTC = startTimeUTC.replace(tzinfo=timezone('UTC'))
            # Convert timezone to Eastern
            startTime = startTimeUTC.astimezone(timezone('US/Eastern'))
            # Split the date and time
            startDate = startTime.strftime("%Y-%m-%d")
            startTime = startTime.strftime("%H:%M")
        except ValueError:
            startDate = startTimeUTC
            startTime = 'TBD'
        
        # Create list to add to dataframe
        nba_games_list = [gameId, seasonYear, league, homeTeamId, awayTeamId, startDate, startTime, status, 
                          homeScore, awayScore]
        
        # Append the list to the dataframe         
        NBA_Games.loc[len(NBA_Games)] = nba_games_list
        
    else:
        continue
        
NBA_Games.head()

,gameId,seasonYear,League,homeTeamId,awayTeamId,startDate,startTime,status,homeScore,awayScore
0,10796,2021,standard,17,4,2021-10-03,15:30,Finished,97,123
1,10797,2021,standard,38,27,2021-10-04,19:00,Finished,123,107
2,10798,2021,standard,2,26,2021-10-04,19:30,Finished,98,97
3,10799,2021,standard,20,1,2021-10-04,19:30,Finished,125,99
4,10800,2021,standard,22,23,2021-10-04,20:00,Finished,117,114


In [94]:
# Connect to SQLite
conn = sqlite3.connect('Data/PropAnalysis.db')

# Create the connection to the SQLite database
engine = create_engine('sqlite:///Data/PropAnalysis.db')
sqlite_connection = engine.connect()

# Save NBA_Games dataframe to SQLite
sqlite_table = "NBA_Games"
NBA_Games.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

# Close connections when done
conn.close()
sqlite_connection.close()

2022-01-18 14:13:08,697 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("NBA_Games")
2022-01-18 14:13:08,698 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-18 14:13:08,700 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("NBA_Games")
2022-01-18 14:13:08,700 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-18 14:13:08,702 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-01-18 14:13:08,702 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-18 14:13:08,704 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("NBA_Games")
2022-01-18 14:13:08,705 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-18 14:13:08,707 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-01-18 14:13:08,708 INFO sqlalchemy.engine.Engine [raw sql] ('NBA_Games',)
2022-01-18 14:13:08,710 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("

#### Update NBA Game Stats Table 

In [95]:
# Get a list of gameIds for completed games
completedGameIds = set(NBA_Games[NBA_Games['status'] == 'Finished']['gameId'].tolist())

# Find a list of gameIds already in NBA_GameStats
NBA_GameStats_Games = set(NBA_GameStats['gameId'].tolist())

# Set the list of gameIds to search to only those completed games that we don't already have data for in NBA_GameStats
gameIds = [x for x in completedGameIds if x not in NBA_GameStats_Games]

# Limited to 100 requests per day before getting charged
counter = 0
limit = 100

gameIds

['10220',
 '10229',
 '10223',
 '10224',
 '10225',
 '10222',
 '10230',
 '10228',
 '10221',
 '10227',
 '10226']

In [96]:
# Create a new NBA_GameStats dataframe that we'll use to append SQLite3 table
NBA_GameStats = pd.DataFrame(columns=['gameId', 'playerId', 'teamId', 'Points', 'Position', 'Minutes', 'FGM', 'FGA',
                                     'FGP', 'FTM', 'FTA', 'TPM', 'TPA', 'TPP', 'offReb', 'defReb', 'totReb', 'assists',
                                      'pFouls', 'steals', 'turnovers', 'blocks', 'plusMinus'])


# Loop over the gameIds and hit RapidAPI for the stats
for game in gameIds:
    print(game)
    
    # Limited to 10 requests per minute, so add a lag
    time.sleep(6)

    # Increment the counter
    counter += 1

    # Hit RapidAPI for this game
    url = f"https://api-nba-v1.p.rapidapi.com/statistics/players/gameId/{game}"

    headers = {
        'x-rapidapi-host': "api-nba-v1.p.rapidapi.com",
        'x-rapidapi-key': "40cfc5891cmshe48d38938873f74p12526ejsnb332fefe9905"
        }

    response = requests.request("GET", url, headers=headers)


    # Set the response as a JSON and extract the games section
    gameStats = json.loads(response.text)
    try:
        gameStats = gameStats['api']['statistics']
    except KeyError:
        print(gameStats)
        continue

    # Loop over the stats and parse needed fields
    for player in gameStats:

        # Grab needed fields
        gameId = player['gameId']
        playerId = player['playerId']
        teamId = player['teamId']
        Points = player['points']
        Position = player['pos']
        Minutes = player['min']
        FGM = player['fgm']
        FGA = player['fga']
        FGP = player['fgp']
        FTM = player['ftm']
        FTA = player['fta']
        TPM = player['tpm']
        TPA = player['tpa']
        TPP = player['tpp']
        offReb = player['offReb']
        defReb = player['defReb']
        totReb = player['totReb']
        assists = player['assists']
        pFouls = player['pFouls']
        steals = player['steals']
        turnovers = player['turnovers']
        blocks = player['blocks']
        plusMinus = player['plusMinus']

        # Create list to add to dataframe
        nba_playerStats_list = [gameId, playerId, teamId, Points, Position, Minutes, FGM, FGA, FGP, FTM, FTA, TPM, TPA, 
                                TPP, offReb, defReb, totReb, assists, pFouls, steals, turnovers, blocks, plusMinus]

        # Append the list to the dataframe         
        NBA_GameStats.loc[len(NBA_GameStats)] = nba_playerStats_list
        
NBA_GameStats.head()

10220
10229
10223
10224
10225
10222
10230
10228
10221
10227
10226


,gameId,playerId,teamId,Points,Position,Minutes,FGM,FGA,FGP,FTM,...,TPP,offReb,defReb,totReb,assists,pFouls,steals,turnovers,blocks,plusMinus
0,10220,941,5,38,SF,38:56,14,20,70.0,5,...,55.6,1,11,12,5,3,1,1,1,26
1,10220,227,5,16,PF,36:21,6,16,37.5,1,...,37.5,0,2,2,4,3,2,2,0,19
2,10220,426,5,5,C,28:24,2,2,100,1,...,0.0,0,2,2,3,4,1,1,0,16
3,10220,1879,5,9,SG,45:33,3,11,27.3,1,...,40.0,2,5,7,3,0,0,1,1,6
4,10220,458,5,22,PG,35:14,8,18,44.4,2,...,36.4,0,6,6,7,0,1,0,0,22


In [97]:
# Connect to SQLite
conn = sqlite3.connect('Data/PropAnalysis.db')

# Create the connection to the SQLite database
engine = create_engine('sqlite:///Data/PropAnalysis.db')
sqlite_connection = engine.connect()

# Save NBA_GameStats dataframe to SQLite
sqlite_table = "NBA_GameStats"
NBA_GameStats.to_sql(sqlite_table, sqlite_connection, if_exists='append')

# Close connections when done
conn.close()
sqlite_connection.close()

2022-01-18 14:15:22,279 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("NBA_GameStats")
2022-01-18 14:15:22,280 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-18 14:15:22,285 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-18 14:15:22,291 INFO sqlalchemy.engine.Engine INSERT INTO "NBA_GameStats" ("index", "gameId", "playerId", "teamId", "Points", "Position", "Minutes", "FGM", "FGA", "FGP", "FTM", "FTA", "TPM", "TPA", "TPP", "offReb", "defReb", "totReb", assists, "pFouls", steals, turnovers, blocks, "plusMinus") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-01-18 14:15:22,292 INFO sqlalchemy.engine.Engine [generated in 0.00401s] ((0, '10220', '941', '5', '38', 'SF', '38:56', '14', '20', '70.0', '5', '5', '5', '9', '55.6', '1', '11', '12', '5', '3', '1', '1', '1', '26'), (1, '10220', '227', '5', '16', 'PF', '36:21', '6', '16', '37.5', '1', '3', '3', '8', '37.5', '0', '2', '2', '4', '3', '2', '2', '0', '19'), (2, '10220', '426', '5',

### Link NBA Stats to Projections and Lines 

In [131]:
# Connect to SQLite
conn = sqlite3.connect('Data/PropAnalysis.db')

# Query bet_offers_df table into dataframe
bet_offers_df = pd.read_sql_query("SELECT * FROM bet_offers_df", conn)
# Query NBA_GameStats table into dataframe
NBA_GameStats = pd.read_sql_query("SELECT * FROM NBA_GameStats", conn)

# Close connection when done
conn.close()

# Reset the index of the table
bet_offers_df.reset_index(drop=True, inplace=True)
# Replace NaN values with NA
bet_offers_df.NBA_Stat.fillna('NA', inplace=True)

# Print the table and number of rows
print(len(bet_offers_df))
bet_offers_df.tail()

1263


,level_0,index,League,Game,StartDate,StartTime,Player,BetLabel,SportsLine_Projection,DK_Line,...,Outcome2_Odds,Line2ProjDiff,Outcome1_ImpliedProbability,Outcome2_ImpliedProbability,Bet_Juice,NBA_Stat,BetSuggestion,BetResult,W/L,BetGrade
1258,1258,97,NBA,BKN Nets @ WAS Wizards,"Wed, Jan 19",07:10 PM,Spencer Dinwiddie,Assists + Rebounds,8.6,9.5,...,-105,0.9,56,51.0,56.0,NA,Under,None,L,F
1259,1259,98,NBA,BKN Nets @ WAS Wizards,"Wed, Jan 19",07:10 PM,Spencer Dinwiddie,Points + Assists,20.1,19.5,...,-115,0.6,53,53.0,53.0,NA,Over,None,L,F
1260,1260,99,NBA,BKN Nets @ WAS Wizards,"Wed, Jan 19",07:10 PM,Kentavious Caldwell-Pope,Assists + Rebounds,4.3,4.5,...,120,0.2,62,45.0,62.0,NA,Under,None,L,F
1261,1261,100,NBA,BKN Nets @ WAS Wizards,"Wed, Jan 19",07:10 PM,Kentavious Caldwell-Pope,Points + Assists,11.2,13.5,...,-125,2.3,52,56.0,52.0,NA,Under,None,L,C
1262,1262,101,NBA,BKN Nets @ WAS Wizards,"Wed, Jan 19",07:10 PM,Kentavious Caldwell-Pope,Points + Rebounds,12.5,14.5,...,-110,2.0,56,52.0,56.0,NA,Under,None,L,D


In [132]:
# Link the NBA stat fields with the bet label tracked
stat_label_map = {
    'Assists': 'assists',
    'Points': 'Points',
    'Rebounds': 'totReb'
}

stat_label_keys = list(stat_label_map.keys())

# Loop over the rows of the bet_offers_df and pull in NBA game stats for that player
for i, row in bet_offers_df.iterrows():

    # Extract needed fields
    Game = row['Game']
    startDate = row['StartDate']
    startTime = row['StartTime']
    Player = row['Player']
    try:
        playerId = NBA_Players.loc[NBA_Players["fullName"] == Player, "playerId"].values[0]
    except:
        playerId = 0
    NBA_Stat = row['NBA_Stat']
    # Find the betLabel and split into component parts, if possible
    betLabel = row['BetLabel']    
    bet_parts = betLabel.split("+")
    bet_parts = [x.strip() for x in bet_parts]
    bet_parts.sort()
    # Find the team Ids
    hTeamId, aTeamId = findTeamIds(Game)
    # Find the gameId 
    gameId = findGameId(Game, startDate, startTime)

    # Find the NBA Stats if the NBA_Stat field is NA and gameId and playerId are found
    if (playerId != 0) & (gameId != 0) & (NBA_Stat == 'NA'):

        # Extract the NBA Stats from the NBA_GameStats table based off of the betLabel
        if len(bet_parts) == 3:

            # Check that the labels are those in the stat_label_map 
            # If not this is likely due to a miss in the Player matching that results in overlap in the BetLabel
            if list(bet_parts) == stat_label_keys:

                # Create a variable to store the NBA Stat
                NBA_Stat = 0

                # Loop over the bet parts and add the projections
                for part in bet_parts:

                    try:
                        # Use the player name and the statistic to look up the value in the NBA_GameStats table
                        stat = NBA_GameStats.loc[(NBA_GameStats.gameId == gameId) & (NBA_GameStats.playerId == playerId),
                                                stat_label_map[part]].iloc[0]
                        # Add the projection to the total
                        NBA_Stat += int(stat)
                    except:
                        NBA_Stat = 'NA'

                # Set the stat in the bet_offers_dailydf table
                bet_offers_df.loc[i, "NBA_Stat"] = NBA_Stat

        elif len(bet_parts) == 2:

            # Check that both the labels are in the stat_label_lookup
            # If not this is likely due to a miss in the Player matching that results in overlap in the BetLabel
            if (bet_parts[0] in stat_label_keys) & (bet_parts[1] in stat_label_keys) :

                # Create a variable to store the NBA Stat
                NBA_Stat = 0

                # Loop over the bet parts and add the projections
                for part in bet_parts:
                    try:
                        # Use the player name and the statistic to look up the value in the NBA_GameStats table
                        stat = NBA_GameStats.loc[(NBA_GameStats.gameId == gameId) & (NBA_GameStats.playerId == playerId),
                                                stat_label_map[part]].iloc[0]
                        # Add the projection to the total
                        NBA_Stat += int(stat)
                    except:
                        NBA_Stat = 'NA'

                # Set the stat in the bet_offers_dailydf table
                bet_offers_df.loc[i, "NBA_Stat"] = NBA_Stat

        else:

            if betLabel in stat_label_map.keys():
                try:
                    # Use the player name and the statistic to look up the value in the NBA_GameStats table
                    NBA_Stat = int(NBA_GameStats.loc[(NBA_GameStats.gameId == gameId) & (NBA_GameStats.playerId == playerId),
                                            stat_label_map[betLabel]].iloc[0])
                except:
                    NBA_Stat = 'NA'
                # Set the stat in the bet_offers_dailydf table
                bet_offers_df.loc[i, "NBA_Stat"] = NBA_Stat
        
        print(i, Player, playerId, startDate, gameId, bet_parts, NBA_Stat)
        
    else:

        continue

942 Amir Coffey 2371 Mon, Jan 17 10224 ['Assists', 'Rebounds'] NA
943 Amir Coffey 2371 Mon, Jan 17 10224 ['Points', 'Rebounds'] NA
944 Amir Coffey 2371 Mon, Jan 17 10224 ['Assists', 'Points'] NA
957 Oshae Brissett 2399 Mon, Jan 17 10224 ['Assists', 'Rebounds'] NA
958 Oshae Brissett 2399 Mon, Jan 17 10224 ['Points', 'Rebounds'] NA
959 Oshae Brissett 2399 Mon, Jan 17 10224 ['Assists', 'Points'] NA
1017 John Konchar 2365 Mon, Jan 17 10223 ['Points', 'Rebounds'] NA
1018 John Konchar 2365 Mon, Jan 17 10223 ['Assists', 'Rebounds'] NA
1019 John Konchar 2365 Mon, Jan 17 10223 ['Assists', 'Points'] NA
1074 Max Strus 2407 Mon, Jan 17 10227 ['Points', 'Rebounds'] NA
1075 Max Strus 2407 Mon, Jan 17 10227 ['Assists', 'Rebounds'] NA
1076 Max Strus 2407 Mon, Jan 17 10227 ['Assists', 'Points'] NA
1086 Luguentz Dort 2322 Mon, Jan 17 10228 ['Assists', 'Points'] NA
1087 Luguentz Dort 2322 Mon, Jan 17 10228 ['Assists', 'Rebounds'] NA
1088 Luguentz Dort 2322 Mon, Jan 17 10228 ['Points', 'Rebounds'] NA
1161

### Grade the Bets versus the Projections

In [124]:
# Format columns
bet_offers_df['NBA_Stat'] = pd.to_numeric(bet_offers_df.loc[:,'NBA_Stat'], errors='coerce')
# Create a column for the bet suggestion based off the SportsLine_Projection and DK_Line
bet_offers_df.loc[bet_offers_df['SportsLine_Projection'] > bet_offers_df['DK_Line'], 'BetSuggestion'] = 'Over'
bet_offers_df.loc[bet_offers_df['SportsLine_Projection'] == bet_offers_df['DK_Line'], 'BetSuggestion'] = 'Push'
bet_offers_df.loc[bet_offers_df['SportsLine_Projection'] < bet_offers_df['DK_Line'], 'BetSuggestion'] = 'Under'
# Create a column for the bet result based off the DK_Line and NBA_Stat
bet_offers_df.loc[bet_offers_df['NBA_Stat'] > bet_offers_df['DK_Line'], 'BetResult'] = 'Over'
bet_offers_df.loc[bet_offers_df['NBA_Stat'] == bet_offers_df['DK_Line'], 'BetResult'] = 'Push'
bet_offers_df.loc[bet_offers_df['NBA_Stat'] < bet_offers_df['DK_Line'], 'BetResult'] = 'Under'
# Create a column for whether the bet was "won" or "lost"
bet_offers_df.loc[bet_offers_df['BetSuggestion'] == bet_offers_df['BetResult'], 'W/L'] = 'W'
bet_offers_df.loc[bet_offers_df['BetSuggestion'] != bet_offers_df['BetResult'], 'W/L'] = 'L'
# Create a column with a "bet grade" for the bet based off the Line2ProjDiff
bet_offers_df['BetGrade'] = bet_offers_df['EstimatedEdge'].apply(betGrade)

In [164]:
# Connect to SQLite
conn = sqlite3.connect('Data/PropAnalysis.db')

# Create the connection to the SQLite database
engine = create_engine('sqlite:///Data/PropAnalysis.db')
sqlite_connection = engine.connect()

# Save NBA_GameStats dataframe to SQLite
sqlite_table = "bet_offers_df"
bet_offers_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

# Close connections when done
conn.close()
sqlite_connection.close()

In [170]:
# Replace NaN values with NA
bet_offers_df.NBA_Stat.fillna('NA', inplace=True)
# Drop rows that we couldn't get an NBA_Stat for 
df = bet_offers_df.loc[BetOdds['NBA_Stat'] != 'NA']
# Create a table to show the win results by bet grade
df_counts = df.groupby(['BetGrade', 'W/L'])['W/L'].size()
df_pcts = df_counts.groupby(level=0).apply(lambda x: round(100 * x / float(x.sum()),1))
# Add the counts back to the table
df = pd.concat([df_counts, df_pcts], axis = 1)
# Format the columns
df.columns = ['Count', 'Win_Pct']
df

Count  Win_Pct
BetGrade W/L                
A        L        8     20.5
         W       31     79.5
B        L       36     36.7
         W       62     63.3
C        L      133     48.9
         W      139     51.1
D        L      139     44.8
         W      171     55.2
F        L      229     54.5
         W      191     45.5